### Record an audio to file

In [2]:
import os
import sounddevice as sd
#import soundfile as sf
import numpy as np
import vlc
from scipy.io.wavfile import write
sr = 16000  # Sample rate
seconds = 3  # Duration of recording
filename = 'myfile.wav'

ModuleNotFoundError: No module named 'sounddevice'

In [104]:
bytearr = sd.rec(int(seconds * fs), samplerate=sr, channels=1)
sd.wait()  # Wait until recording is finished
# Convert `data` to 16 bit integers:
y = (np.iinfo(np.int16).max * (bytearr/np.abs(data).max())).astype(np.int16) 
wavfile.write(filename, fs, y)

In [3]:
# Playback the file
def play_file(filename, sr):
    data, fs = sf.read(filename)  
    sd.play(data, sr)
    status = sd.wait()  # Wait until file is done playing
def play_mp3(filename):
    p = vlc.MediaPlayer(filename)
    p.play()

In [4]:
from pydub import AudioSegment
from pydub.utils import mediainfo
def mp3_to_wav(infile, outfile):
    sr = mediainfo(infile)['sample_rate']
    sound = AudioSegment.from_mp3(infile)
    sound.export(outfile, format="wav")
    return sr

### Get a German transcript from Google ASR

In [4]:
from google.cloud import speech
import io

In [5]:
credentials='/home/felix/data/research/Google/AffectiveAgents-7874f11e65d6.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=credentials

In [107]:
client = speech.SpeechClient()
with io.open(filename, "rb") as audio_file:
    content = audio_file.read()
audio = speech.RecognitionAudio(content = content)
config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    language_code="de-DE",
)

In [108]:
response = client.recognize(config=config, audio=audio)
for result in response.results:
    for index, alternative in enumerate(result.alternatives):
        print("Transcript {}: {}".format(index, alternative.transcript))

Transcript 0: AstraZeneca in Baden-Württemberg


### Playback synthesized text with Google

In [ ]:
text = 'In {} sind {} Menschen mit {} geimpft.'.format('Berlin', 123, 'Biontech')

In [5]:
from gtts import gTTS
tts = gTTS(text, lang='de')
audio_file = './out_synth.mp3'
tts.save(audio_file)
sr = mp3_to_wav(audio_file, 'test.wav')
play_file('test.wav', int(sr))

### Playback with pytts (using espeak)

In [31]:
import pyttsx3
engine = pyttsx3.init()
engine.setProperty('voice', 'german')
engine.setProperty('rate', 300)
engine.say(text)
engine.runAndWait()

In [21]:
voices = engine.getProperty('voices') 
for v in voices:
    print(v)

### Voxpopuli (using mbrola)

In [11]:
from voxpopuli import Voice
voice = Voice(lang="de", pitch=50, speed=200, voice_id=6)
voice.say(text)